In [70]:
# Libraries
import os
import re
import json
import folium
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


api_key = "768d81e0c5c733e2a375488139b78bb0"

baseGeoUrl = f"http://api.openweathermap.org/geo/1.0/direct?q=London&limit=1&appid={api_key}"
geoResponse = urlopen(baseGeoUrl)

geo_orig = json.loads(geoResponse.read())
geo_df = pd.DataFrame(geo_orig)
# lat and lon of city

lat = geo_df.loc[0, "lat"]
lon = geo_df.loc[0, "lon"]

# Get current weather data
weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
response = urlopen(weather_url)
weather_data = json.loads(response.read())

# Extract relevant information
current_temperature = weather_data["main"]["temp"]
max_temperature = weather_data["main"]["temp_max"]
min_temperature = weather_data["main"]["temp_min"]
sunrise = pd.to_datetime(weather_data["sys"]["sunrise"], unit="s")
sunset = pd.to_datetime(weather_data["sys"]["sunset"], unit="s")

# Create a DataFrame
data = {
    "Current Temperature": [current_temperature],
    "Max Temperature": [max_temperature],
    "Min Temperature": [min_temperature],
    "Sunrise": [sunrise],
    "Sunset": [sunset]
}
df = pd.DataFrame(data)

# Print the DataFrame
df




,Current Temperature,Max Temperature,Min Temperature,Sunrise,Sunset
0,286.21,287.38,284.59,2023-05-17 04:06:25,2023-05-17 19:47:35
